# Redes bayesianas - Resident Evil

In [122]:
import networkx  # Permite trabajar con grafos
import pgmpy.models as pgmm  # Modelos gráficos de probabilidad
import pgmpy.factors as pgmf  # Tablas de probabilidades condicionales y
                              # potenciales de probabilidad
import pgmpy.inference as pgmi  # Inferencia probabilística exacta

In [126]:
import pgmpy.readwrite.XMLBIF as rwBIF

reader = rwBIF.XMLBIFReader('residentevil.xml')
Modelo_residentevil = reader.get_model()

In [145]:
Modelo_residentevil.cpds

[<TabularCPD representing P(Lugar:3) at 0x1199dc160>,
 <TabularCPD representing P(Ciudad:2 | Lugar:3) at 0x1199dc908>,
 <TabularCPD representing P(Horda:2 | Ciudad:2, Lugar:3) at 0x1199dc128>,
 <TabularCPD representing P(Medicina:2 | Ciudad:2) at 0x1199dc860>,
 <TabularCPD representing P(Agua:2 | Ciudad:2) at 0x1199dc0b8>,
 <TabularCPD representing P(Comida:2 | Ciudad:2) at 0x1199dc5c0>,
 <TabularCPD representing P(Refugio:2 | Ciudad:2) at 0x1199dcda0>,
 <TabularCPD representing P(Vehiculo:2 | Ciudad:2) at 0x1199dc6d8>,
 <TabularCPD representing P(Armas:2 | Ciudad:2) at 0x1199dc748>,
 <TabularCPD representing P(Grupo:2 | Ciudad:2) at 0x1199dc7b8>,
 <TabularCPD representing P(Soldados umbrella:2 | Ciudad:2) at 0x1199dcb70>,
 <TabularCPD representing P(Cooperativo:2 | Refugio:2, Grupo:2) at 0x1199dca58>]

In [128]:
#Función que descarta las variables irrelevantes

def descarta(m, c, e):
    lista = c+e
    descartados = []
    padres = []
    for l in lista:
        subgrafo = Modelo_residentevil.subgraph(networkx.ancestors(Modelo_residentevil, l))
        for s in subgrafo:
            padres.append(s)
                          
    for n in m.nodes():
        if n not in lista and n not in padres:
            descartados.append(n)
                  
    return descartados  

consulta = []
consulta.append("Refugio")
evidencias = []
evidencias.append("Cooperativo")
evidencias.append("Medicina")
descarta(Modelo_residentevil,consulta,evidencias)

['Horda', 'Agua', 'Comida', 'Vehiculo', 'Armas', 'Soldados umbrella']

In [ ]:
#Función que devuelve los factores iniciales

def factores_iniciales(m):
    d = descarta(m,consulta,evidencias)
    c = 0
    factores = []
    for n in m.nodes():
        if n not in d:
            factores.append(m.cpds[c].to_factor())
        c+=1
    return factores


In [148]:
#2) Eliminación de variables: Uso de la clase VariableElimination
from pgmpy.inference import VariableElimination
infer = VariableElimination(Modelo_residentevil)
print(infer.query(['Refugio']) ['Refugio'])
    

/Users/anapareciendo/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


ValueError: Sum of probabilites of states for node Ciudad is not equal to 1.